## fetching hot company

In [1]:
from collections import Counter
from NCHU_nlptoolkit.cut import *
import requests
from text_cleaner.text_cleaner import Text_Cleaner
import json

[nltk_data] Downloading package punkt to /user_data/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /user_data/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /user_data/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /user_data/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to /user_data/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Building prefix dict from /user_data/.local/lib/python3.10/site-packages/NCHU_nlptoolkit/dictionary/dict.txt.big.txt ...
Loading model from cache /tmp/jieba.u26cdf4ccd330cf5adf1c104d33cbd81f.cache
Loading model cost 0.239 seconds.
Prefix dict has been built successfully.


In [85]:
K = 10 # for company hot today
news_K = 5 # for company's news
start = 1704297600
end = 1704384000
size = 10000
page = 0
news_period = 2 # fetch news day period
url = f"https://ptt-search.nlpnchu.org/api/GetByBoard?board=Stock&start={start}&end={end}&size={size}&from={page}"

f = requests.get(url)
res = f.json()

tc = Text_Cleaner()

In [6]:
all_contents = []

for content in res['hits']:
    all_contents += list(cut_sentence(tc.clean_text(content['_source']['content']), minword=2))
    
print(len(all_contents))

27697


In [7]:
frequency_counting_table = dict(Counter(all_contents))
frequency_counting_table = sorted(frequency_counting_table.items(), key=lambda x:x[1], reverse = 1)
print(frequency_counting_table)

[('長榮', 171), ('Etf', 146), ('台股', 141), ('航運', 133), ('陽明', 115), ('明天', 101), ('應該', 93), ('去年', 82), ('萬海', 80), ('營收', 78), ('Ai', 77), ('股票', 74), ('一堆', 73), ('原文', 73), ('股價', 70), ('羨慕', 70), ('賺錢', 69), ('上船', 66), ('空軍', 66), ('股息', 64), ('笑死', 63), ('時間', 63), ('今年', 62), ('Qq', 61), ('看到', 59), ('已經', 59), ('美國', 53), ('畢業', 53), ('來了', 53), ('昨天', 51), ('有人', 50), ('了嗎', 50), ('比較', 49), ('投信', 49), ('獲利', 48), ('公司', 48), ('進場', 48), ('中國', 47), ('市場', 47), ('準備', 46), ('老謝', 46), ('貨櫃', 45), ('外資', 45), ('股市', 45), ('感謝', 45), ('紅海', 45), ('直接', 44), ('運價', 44), ('不用', 44), ('大盤', 43), ('船票', 42), ('美債', 41), ('水手', 41), ('賠錢', 40), ('大哥', 40), ('還在', 39), ('這種', 39), ('指數', 38), ('Cc', 38), ('謝謝', 37), ('是不是', 37), ('成長', 37), ('標題', 36), ('億元', 36), ('訊號', 36), ('資金', 35), ('Pc', 35), ('散戶', 35), ('問題', 34), ('每天', 34), ('之前', 34), ('元大', 34), ('美股', 34), ('海運', 34), ('根本', 33), ('夜盤', 33), ('持股', 33), ('美元', 33), ('自營', 33), ('出貨', 33), ('高鐵', 32), ('開盤', 32), ('一年', 

In [9]:
company_file = open('/user_data/project/PTTSocailEngine/backend/data/上市公司基本資料.json')
all_company = json.load(company_file)

In [11]:
def load_complexity_companies():
    with open("/user_data/project/PTTSocailEngine/backend/data/易混淆公司.txt", 'r', encoding='utf-8') as file:
        complexity_companies = set(word.strip() for word in file)
    return complexity_companies


complexity_companies = load_complexity_companies()

In [12]:
def search_in_all_company(value: str):
    for company in all_company:
        if value not in complexity_companies:
            if value == company['公司代號'] or value in company['公司簡稱']:
                # print(f"value is {value}, company['公司代號'] is {company['公司代號']}, company['公司簡稱'] is {company['公司簡稱']}")
                return company['公司簡稱']
        else:
            if value == company['公司代號'] or value == company['公司簡稱']:
                # print(f"value is {value}, company['公司代號'] is {company['公司代號']}, company['公司簡稱'] is {company['公司簡稱']}")
                return company['公司簡稱']

In [13]:
def filt_company(topK: list):
    companines = []
    for K in topK:
        company_name = search_in_all_company(K[0])
        if company_name:
            new_tuple = (company_name, K[1])
            companines.append(new_tuple)

    return companines

In [14]:
hot_company_today = filt_company(frequency_counting_table)[0:K]
print(hot_company_today)
print(len(hot_company_today))

[('長榮', 171), ('陽明', 115), ('萬海', 80), ('元大金', 34), ('台灣高鐵', 32), ('全新', 31), ('華城', 31), ('南電', 31), ('長榮航', 30), ('國泰金', 20)]
10


## fetching Google News

In [54]:
from GoogleNews import GoogleNews

def get_all_news(company):
  googlenews = GoogleNews(lang='zh-TW', encode='utf-8', period=f'{news_period}d')
  googlenews.enableException(True)
  googlenews.get_news(company)
  all_news = googlenews.results(sort=True)
  googlenews.clear()
  return all_news

In [77]:
import time
from bs4 import BeautifulSoup

def fetch_company_news_content(company_news):
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                                'Chrome/87.0.4280.141 Safari/537.36'}
    
    for news in company_news:
        time.sleep(0.5)
        contents = ""
        url = news['link']
        # print(f'url is {url}')
        response = requests.get(f"https://{url}", headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        newsUrl = soup.find_all('c-wiz', class_='qFYwOb')[0].find('a').getText()

        # 取得該篇新聞連結內容
        response = requests.get(newsUrl, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser') 
        all_p = soup.find_all('p')

        for p in all_p:
            contents += p.getText().strip()

        news['content'] = contents

    return company_news


In [78]:
def fetch_company_info(company, freq):
    company_news_K = get_all_news(company=company)[0:news_K]
    print("="*80)
    print(f"company is {company} frequency is {freq}\n Fetching...")
    company_news_K = fetch_company_news_content(company_news_K)
    print("="*80)
    return company_news_K

In [86]:
all_company_info = []
for company, freq in hot_company_today:
    all_company_info.append(fetch_company_info(company, freq))
    

company is 長榮 frequency is 171
 Fetching...
company is 陽明 frequency is 115
 Fetching...
company is 萬海 frequency is 80
 Fetching...
company is 元大金 frequency is 34
 Fetching...
company is 台灣高鐵 frequency is 32
 Fetching...
company is 全新 frequency is 31
 Fetching...
company is 華城 frequency is 31
 Fetching...
company is 南電 frequency is 31
 Fetching...
company is 長榮航 frequency is 30
 Fetching...
company is 國泰金 frequency is 20
 Fetching...


In [87]:
def save_hot_company_today(hot_company_today):
    with open("/user_data/project/PTTSocailEngine/backend/data/hot_company_today.json", 'w', encoding='utf-8') as file:
        json.dump(hot_company_today, file, ensure_ascii=False)

save_hot_company_today(all_company_info)

## Testing Block

### TODO: filt complexity company

### Test GoogleNews finance

In [50]:
from GoogleNews import GoogleNews
googlenews = GoogleNews(lang='zh-TW', encode='utf-8', period='1d')
googlenews.enableException(True)
googlenews.get_news('長榮')
all_news = googlenews.results(sort=True)

In [51]:
all_news[0]['link']

'news.google.com/articles/CBMi4wFodHRwczovL3R3Lm5ld3MueWFob28uY29tLyVFNiVBOSU5RiVFNSVCOCVBQiVFNyVCRCVCNyVFNSVCNyVBNSVFNSVBMiU5RSVFNSU4OCU5Ny0lRTklOUQlOUUlRTklOTUlQjclRTYlQTYlQUUlRTYlOUMlODMlRTUlOTMlQTElRTYlOEElOTUlRTclQTUlQTgtJUU2JTk4JThFJUU5JTk2JThCJUU3JUE1JUE4JUU2JTlBJUFCJUU3JUI3JUE5JUU1JUJCJUI2JUU4JTg3JUIzMS0yMi0wOTU4MTEwNTAuaHRtbNIB6wFodHRwczovL3R3Lm5ld3MueWFob28uY29tL2FtcGh0bWwvJUU2JUE5JTlGJUU1JUI4JUFCJUU3JUJEJUI3JUU1JUI3JUE1JUU1JUEyJTlFJUU1JTg4JTk3LSVFOSU5RCU5RSVFOSU5NSVCNyVFNiVBNiVBRSVFNiU5QyU4MyVFNSU5MyVBMSVFNiU4QSU5NSVFNyVBNSVBOC0lRTYlOTglOEUlRTklOTYlOEIlRTclQTUlQTglRTYlOUElQUIlRTclQjclQTklRTUlQkIlQjYlRTglODclQjMxLTIyLTA5NTgxMTA1MC5odG1s?hl=zh-TW&gl=TW&ceid=TW%3Azh-Hant'

In [18]:
from bs4 import BeautifulSoup

headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                             'Chrome/87.0.4280.141 Safari/537.36'}
# resp = requests.get(f"https://{all_news[0]['link']}", headers=headers)
url = all_news[0]['link']

response = requests.get(f"https://{url}", headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
newsUrl = soup.find_all('c-wiz', class_='qFYwOb')[0].find('a').getText()

 # 取得該篇新聞連結內容
response = requests.get(newsUrl, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser') 
all_p = soup.find_all('p')
contents = ""

for p in all_p:
    contents += p.getText().strip()
    
contents


'長榮航空機師指控長榮航空拒絕合理調薪、調整外站津貼，以及違法聘雇外籍機師，日前由桃園市機師工會發起為期2周罷工投票，投至今天下午5時止，原定明開票公布結果。不過，桃園市機師職業工會理事長李信燕今表示，主管機關認為全體工會成員都參與罷工投票，程序才完備，為避免確保相關程序進行，以及避免開票結果互相影響，明原定開票暫緩，預定8日開啟為期兩周第二階段投票，兩階段投票結果均在22日開票。桃園市機師工會今天表示，近日因中央主管機關勞動部及地方主管機關桃園市勞動局對僅由工會長榮分會會員，而非全體會員（包含非長榮分會之其他會員）進行投票表達疑義，傾向建議，即便僅為工會長榮分會會員與長榮航空公司間之勞資爭議，程序上仍亦宜由工會全體會員投票，較為完備。桃園市機師工會表示，經臨時理監事會討論後，尊重主管機關見解，另為萬無一失地確保合法罷工權取得一切相關程序，無任何可置喙、質疑之空間，決議將於在1月8日至21日，增列由機師工會「非長榮分會會員」進行投票程序。桃園市機師工會指出，長榮分會會員投票期間，並未因此而延長，同樣維持1月4日下午5點至。1月4日下午5時後，會在工會監事與律師的見證，以及錄影存證下封存票匭。桃園市機師工會表示，原訂1月5日上午針對長榮分會會員投票結果開票，在避免對後續工會「非長榮分會會員」投票意向產生不必要影響考量下，將延後至1月21日工會「非長榮分會會員」投票時間截止後隔日，亦即1月22日上午，統一開票，並一次公布完整結果。1月22日上午統一開票過程，勞工主管機關亦將派員全程指導、確認程序。▪ 月付2萬房貸想住大樓？月薪6萬者在這2都有機會\n ▪ 打房這麼用力 為何房價不降？賴正鎰：這情況40年沒見\n ▪ 賣厝阿明／買賣成交後 屋主有權不讓買方看屋？\n ▪ 他衝買千萬宅發現慘了 新青安買房月薪得多少？\n ▪ 台中奇葩建築有新朋友「焦阿巴」？ 市府大解謎長榮航空機師指控長榮航空拒絕合理調薪、調整外站津貼，以及違法聘雇外籍機師，日前由桃園市機師工會發起為期2周罷工投票，投至...隨著歐美政府推動網路基礎建設政策明確，擁有全球最多人口的印度市場也將跟進，且在美中維持對抗的態勢之下，將有利台廠持續擴大網通市占率。日本石川縣能登半島地區在1月1日下午發生芮氏規模7.6強震，造成嚴重人員傷亡及重大災情。為協助日本賑災，外交部今（4）日...台南市去年10月住宅價格指數